In [1]:
import pandas as pd
import numpy as np

In [2]:
def downsample_gps_by_order(df: pd.DataFrame, rule: str = '1T') -> pd.DataFrame:
    """
    对GPS轨迹数据按订单进行时间降采样。

    Args:
        df (pd.DataFrame): 包含GPS数据的DataFrame，
                           必须包含 ['order_id', 'gps_time', 'longitude', 'latitude'] 字段。
                           'driver_id' 字段也会被保留。
        rule (str, optional): 降采样的时间频率规则。
                              默认为 '1T' (即 '1min'，每分钟采样一次)。
                              常用规则: '5S' (5秒), '10T' (10分钟), 'H' (每小时)。

    Returns:
        pd.DataFrame: 降采样后的新DataFrame。
    """
    # 0. 复制DataFrame以避免修改原始数据
    df_copy = df.copy()

    # 1. 确保 'gps_time' 是 datetime 类型，这是进行时间操作的前提
    # to_datetime 会自动尝试解析多种格式，包括Unix时间戳（如果指定单位）
    if not np.issubdtype(df_copy['gps_time'].dtype, np.datetime64):
        df_copy['gps_time'] = pd.to_datetime(df_copy['gps_time'])

    # 2. 将 'gps_time' 设置为索引，以便使用 resample
    df_copy = df_copy.set_index('gps_time')

    # 3. 创建一个列表来存储每个订单降采样后的结果
    resampled_dfs = []

    # 4. 按 'order_id' 分组并对每个组进行操作
    for order_id, group in df_copy.groupby('order_id'):
        # 对每个订单的DataFrame进行降采样
        # .resample(rule) 创建时间窗口
        # .first() 在每个窗口中取第一条记录
        # 这会保留 driver_id, longitude, latitude 等字段的第一个值
        resampled_group = group.resample(rule).first()

        # .resample() 可能会产生所有列都为NaN的空行（如果某个时间窗口内没有数据）
        # 我们使用 dropna() 清除这些完全为空的行
        resampled_group = resampled_group.dropna(how='all')

        # 将 order_id 加回到列中，因为它在groupby后变成了组的名称
        resampled_group['order_id'] = order_id

        resampled_dfs.append(resampled_group)

    # 5. 将所有处理过的组重新合并成一个DataFrame
    if not resampled_dfs:
        # 如果输入为空或结果为空，返回一个结构相同的空DataFrame
        return pd.DataFrame(columns=df.columns)

    final_df = pd.concat(resampled_dfs)

    # 6. 重置索引，将 'gps_time' 从索引变回普通列
    final_df = final_df.reset_index()

    # 7. 重新排列列的顺序，与输入保持一致
    # 使用 [col for col in df.columns if col in final_df.columns] 来确保列存在
    final_df = final_df[df.columns]

    return final_df

In [3]:
original_df = pd.read_csv('filtered_orders.csv')

In [4]:
# '1T' 或 '1min' 代表1分钟
# ‘5S’ 代表5秒钟
downsampled_df = downsample_gps_by_order(original_df, rule='20s')
print(downsampled_df)
print(f"\n降采样后数据行数: {len(downsampled_df)}")

downsampled_df.to_csv('filtered_downsample.csv', index=False)

                              driver_id                          order_id  \
0      9a9ac4db0843220095674816bcbe28d7  000a7cb982caa7c3a7b0245b6fd3da08   
1      9a9ac4db0843220095674816bcbe28d7  000a7cb982caa7c3a7b0245b6fd3da08   
2      9a9ac4db0843220095674816bcbe28d7  000a7cb982caa7c3a7b0245b6fd3da08   
3      9a9ac4db0843220095674816bcbe28d7  000a7cb982caa7c3a7b0245b6fd3da08   
4      9a9ac4db0843220095674816bcbe28d7  000a7cb982caa7c3a7b0245b6fd3da08   
...                                 ...                               ...   
57780  2eb4c952f0af747168f11dd229936342  fff0ee2db1bd1cd98ce4c49cb0cd4851   
57781  2eb4c952f0af747168f11dd229936342  fff0ee2db1bd1cd98ce4c49cb0cd4851   
57782  2eb4c952f0af747168f11dd229936342  fff0ee2db1bd1cd98ce4c49cb0cd4851   
57783  2eb4c952f0af747168f11dd229936342  fff0ee2db1bd1cd98ce4c49cb0cd4851   
57784  2eb4c952f0af747168f11dd229936342  fff0ee2db1bd1cd98ce4c49cb0cd4851   

                 gps_time   longitude   latitude  time_diff  
0     2016-11

In [10]:
def add_gps_noise(df: pd.DataFrame, noise_level_meters: float = 10.0) -> pd.DataFrame:
    """
    为DataFrame中的GPS坐标点添加高斯噪声。

    该函数会创建新的 'longitude_noisy' 和 'latitude_noisy' 列。

    Args:
        df (pd.DataFrame): 包含GPS数据的DataFrame，
                           必须包含 ['longitude', 'latitude'] 字段。
        noise_level_meters (float, optional): 噪声的标准差，以米为单位。
                                              默认为 10.0。这个值越大，噪声点越离散。

    Returns:
        pd.DataFrame: 带有噪声GPS坐标的新列的DataFrame。
    """
    # 0. 复制DataFrame以避免修改原始数据
    df_copy = df.copy()

    # 1. 定义米和度之间的转换常数
    METERS_PER_DEGREE_LAT = 111132.0  # 全局近似值

    # 2. 生成高斯噪声（均值为0，标准差为1）
    # 我们为经度和纬度分别生成与DataFrame行数相同的随机数
    num_rows = len(df_copy)
    # loc=0.0 (均值), scale=1.0 (标准差), size=num_rows (数量)
    random_gaussian = np.random.normal(0.0, 1.0, size=(num_rows, 2))

    # 3. 将噪声从米转换为度
    # 纬度噪声（单位：度）
    lat_noise = (random_gaussian[:, 0] * noise_level_meters) / METERS_PER_DEGREE_LAT

    # 经度噪声（单位：度）
    # 经度的转换依赖于纬度，我们使用向量化操作来为每一行计算其对应的转换因子
    # np.cos() 需要弧度，所以先用 np.radians() 转换
    meters_per_degree_lon = 111320.0 * np.cos(np.radians(df_copy['latitude']))
    lon_noise = (random_gaussian[:, 1] * noise_level_meters) / meters_per_degree_lon

    # 4. 将噪声添加到原始坐标上，创建新列
    df_copy['longitude'] = df_copy['longitude'] + lon_noise
    df_copy['latitude'] = df_copy['latitude'] + lat_noise

    return df_copy

In [11]:
df_with_noise = add_gps_noise(downsampled_df, noise_level_meters=50)

df_with_noise.to_csv('filtered_with_noise.csv', index=False)